**Required Libraries**

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
os.chdir('C:\\Users\HP\\Desktop\\nilesh\\krish practice dataset\\fake news classifier')

**Load Dataset**

In [4]:
df = pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [6]:
# Check target column

df['label'].value_counts()

1    10413
0    10387
Name: label, dtype: int64

In [7]:
df.shape

(20800, 5)

In [8]:
# Check Missing Values

df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
# drop NaN values present in Dataset

df = df.dropna()

In [10]:
# Handle the Missing values present in dataset

df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [11]:
df.shape

(18285, 5)

In [12]:
df = df.reset_index()
df.head()

,index,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [13]:
# Here not required columns like 'id', 'text', and 'author' so we can drop it.

df = df.drop(['id', 'text', 'author'], axis= 1)
df.head()

,index,title,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0
2,2,Why the Truth Might Get You Fired,1
3,3,15 Civilians Killed In Single US Airstrike Hav...,1
4,4,Iranian woman jailed for fictional unpublished...,1


In [14]:
# Get dependent and independent features

X = df.drop(['label'], axis = 1)
y = df['label']

**Data Preprocessing**

In [15]:
import re
import nltk
from nltk.corpus import stopwords

In [16]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout

In [17]:
# define vocabulary size

voc_size = 5000

In [18]:

from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

ps = PorterStemmer()
lm = WordNetLemmatizer()

corpus = []
for i in range(0,len(df)):
    print(i)
    review = re.sub('[^a-zA-Z]',' ', df['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [lm.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [19]:
len(corpus)

18285

In [20]:
df['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [21]:
corpus[0]

'house dem aide even see comey letter jason chaffetz tweeted'

**Embedding Representation**

In [22]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
print(onehot_repr)

[[3583, 4722, 2170, 3450, 3274, 257, 1227, 1656, 3628, 1359], [4765, 3025, 3602, 195, 1004, 4820, 152], [4489, 1374, 4042, 3972], [4820, 4182, 1751, 4715, 2618, 655], [2932, 1004, 2119, 973, 3899, 4062, 1004, 2761, 986, 481], [1262, 3301, 4428, 2062, 4168, 4515, 3838, 782, 3742, 3579, 4050, 4236, 3975, 3153, 152], [1063, 1310, 2427, 3848, 4619, 3873, 384, 4560, 3548, 2956, 1503], [1102, 4120, 3022, 1438, 4829, 4346, 4515, 1652, 3548, 2956, 1503], [1885, 2001, 2612, 204, 3155, 1267, 602, 2317, 4515, 4546], [3892, 1232, 1984, 4096, 4294, 4368, 4125, 710], [291, 4897, 3634, 2030, 4086, 1662, 2875, 766, 540, 2111, 3872], [4715, 2661, 3274, 873, 4515, 4829], [4142, 2045, 4301, 1397, 3728, 53, 4328, 4108, 4441], [452, 1368, 77, 2288, 1731, 1682, 4834, 3548, 2956, 1503], [3960, 552, 811, 4436, 1534, 3548, 2956, 1503], [4162, 2377, 4961, 4476, 1524, 1447, 2337, 3362, 3891, 3036], [4192, 1267, 3025], [2741, 2711, 2272, 4565, 4515, 812, 38, 152], [3599, 2389, 3602, 1990, 2797, 4795, 1901, 37, 40

In [23]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr , padding='pre' ,maxlen = sent_length)
print(embedded_docs)

[[   0    0    0 ... 1656 3628 1359]
 [   0    0    0 ... 1004 4820  152]
 [   0    0    0 ... 1374 4042 3972]
 ...
 [   0    0    0 ... 3548 2956 1503]
 [   0    0    0 ... 3569 4097 2650]
 [   0    0    0 ... 4726 2443 3271]]


In [24]:
len(embedded_docs)

18285

In [25]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3583,
       4722, 2170, 3450, 3274,  257, 1227, 1656, 3628, 1359])

**Data split into Training set and test set**

In [26]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size= 0.20, random_state= 42, stratify= y)

In [28]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(14628, 20) (14628,) (3657, 20) (3657,)


**Model Training**

In [29]:
embedding_vector_features = 40
model = Sequential()

model.add(Embedding(voc_size,embedding_vector_features,input_length = sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss='binary_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.fit(X_train, y_train , validation_data=(X_test,y_test), epochs=10 , batch_size = 64)

Epoch 1/10
229/229 [==============================] - 4s 16ms/step - loss: 0.3264 - accuracy: 0.8438 - val_loss: 0.2017 - val_accuracy: 0.9133
Epoch 2/10
229/229 [==============================] - 3s 13ms/step - loss: 0.1459 - accuracy: 0.9436 - val_loss: 0.2006 - val_accuracy: 0.9100
Epoch 3/10
229/229 [==============================] - 3s 13ms/step - loss: 0.1094 - accuracy: 0.9587 - val_loss: 0.2142 - val_accuracy: 0.9191
Epoch 4/10
229/229 [==============================] - 3s 13ms/step - loss: 0.0818 - accuracy: 0.9704 - val_loss: 0.2375 - val_accuracy: 0.9054
Epoch 5/10
229/229 [==============================] - 3s 14ms/step - loss: 0.0629 - accuracy: 0.9794 - val_loss: 0.2983 - val_accuracy: 0.9229
Epoch 6/10
229/229 [==============================] - 3s 15ms/step - loss: 0.0484 - accuracy: 0.9830 - val_loss: 0.2866 - val_accuracy: 0.9098
Epoch 7/10
229/229 [==============================] - 3s 13ms/step - loss: 0.0373 - accuracy: 0.9874 - val_loss: 0.3460 - val_accuracy: 0.9117

**Performance Metrics And Accuracy**

In [36]:
y_pred = model.predict_classes(X_test)

In [32]:
from sklearn.metrics import confusion_matrix

In [33]:
confusion_matrix(y_test, y_pred)

array([[1895,  177],
       [ 158, 1427]], dtype=int64)

In [34]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9083948591741865

In [35]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.91      0.92      2072
           1       0.89      0.90      0.89      1585

    accuracy                           0.91      3657
   macro avg       0.91      0.91      0.91      3657
weighted avg       0.91      0.91      0.91      3657

